In [1]:
import os

In [2]:
# this makes the parent file the current working directory
# NOTE - Restart whole notebook otherwise the wrong working directory will be set
os.chdir("../")

print(os.getcwd())

/Users/yyh/Documents/selfstudy/E2E-Kidney-Disease-MLOps-Project


In [3]:
import os
import zipfile
import gdown

from dataclasses import dataclass
from pathlib import Path

from KidneyDiseasePrediction import logger
from KidneyDiseasePrediction.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from KidneyDiseasePrediction.utils.common import read_yaml, create_directories, get_size


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


class ConfigManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:

            dataset_url = self.config.source_url
            zip_download_url = self.config.local_data_file

            # why not also use the config path ?
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading dataset from {dataset_url} to {zip_download_url}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(f"{prefix}{file_id}", zip_download_url)

            logger.info(f"Downloaded dataset to {zip_download_url}")

        except Exception as e:
            raise e

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [4]:
# Create a pipline

try:
    # NOTE - figure out if this is necessary
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

2025-07-10 10:39:40,678 - INFO - common - YAML file config/config.yaml loaded successfully.
2025-07-10 10:39:40,679 - INFO - common - YAML file params.yaml loaded successfully.
2025-07-10 10:39:40,680 - INFO - common - Directory artifacts created at artifacts
2025-07-10 10:39:40,680 - INFO - common - Directory artifacts/data_ingestion created at artifacts/data_ingestion
2025-07-10 10:39:40,681 - INFO - 3617860386 - Downloading dataset from https://drive.google.com/file/d/1jsHsEOQWO36My3YWQQ_IbnxCBlRrDQIS/view?usp=sharing to artifacts/data_ingestion/dataset.zip


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1jsHsEOQWO36My3YWQQ_IbnxCBlRrDQIS
From (redirected): https://drive.google.com/uc?export=download&id=1jsHsEOQWO36My3YWQQ_IbnxCBlRrDQIS&confirm=t&uuid=94e75b6a-3c7b-4078-a174-1939190efb78
To: /Users/yyh/Documents/selfstudy/E2E-Kidney-Disease-MLOps-Project/artifacts/data_ingestion/dataset.zip
100%|██████████| 1.63G/1.63G [00:21<00:00, 74.5MB/s]

2025-07-10 10:40:04,670 - INFO - 3617860386 - Downloaded dataset to artifacts/data_ingestion/dataset.zip
